## Preparing SV Dataset

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

#np.set_printoptions(formatter={'int':hex})

import os
files = os.listdir('sv_traces/')

### reads an sv_trace file into 2 structures:
### pcs: a list of 1001 16-bit pc values
### mem: a list of 1001 65542-byte memory vectors
def read_trace(sv_file):
    with open(sv_file, 'rb') as f:
        lines = f.read()
        lines = np.frombuffer(lines, dtype=np.uint8)
        lines = lines.reshape(1001,-1)
        pcs = lines[:, 0:2]
        mem = lines[:, :]
        #pcs = [np.uint16(i[1]<<8 | i[0]) for i in pcs]
        return mem, pcs
    
### reads trace data into a dataset of mem-to-pc mappings, where
### mem_current: partial memory trace &
### pcs_current: full pcs trace
def read_dataset(file):
    mem_trace = dict_traces[file][0]
    mem_current = mem_trace[:]
    pcs_trace = dict_traces[file][1]
    pcs_current = pcs_trace[:]
    return mem_current, pcs_current

### dict_traces[file][0]: memory trace of <file>
### dict_traces[file][1]: pc trace of <file>
dict_traces = {}
for file in files[0:20]:
    dict_traces[file] = read_trace(os.path.join('sv_traces/', file))
    
### dict_dataset[file][0]: memory trace of <file>
### dict_dataset[file][1]: pc trace of <file>
dict_dataset = {}
for trace in dict_traces:
    dict_dataset[trace] = read_dataset(trace)

In [2]:
dict_traces

{'167:149:9.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2,   0,   0],
         [  3,   2,   0, ...,   2,   0,   0],
         ...,
         [  3,   2, 126, ...,   2,   0,   0],
         [  5,   2,  20, ...,   2,   0,   0],
         [  7,   2,  20, ...,   2,   0,   0]], dtype=uint8),
  array([[0, 2],
         [1, 2],
         [3, 2],
         ...,
         [3, 2],
         [5, 2],
         [7, 2]], dtype=uint8)),
 '121:187:13.svbinttrc': (array([[ 0,  2,  0, ...,  2,  0,  0],
         [ 1,  2,  0, ...,  2,  0,  0],
         [ 3,  2,  0, ...,  2,  0,  0],
         ...,
         [ 9,  2, 50, ...,  2,  0,  0],
         [10,  2, 50, ...,  2,  0,  0],
         [11,  2, 50, ...,  2,  0,  0]], dtype=uint8),
  array([[ 0,  2],
         [ 1,  2],
         [ 3,  2],
         ...,
         [ 9,  2],
         [10,  2],
         [11,  2]], dtype=uint8)),
 '144:101:13.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2

In [3]:
dict_dataset

{'167:149:9.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2,   0,   0],
         [  3,   2,   0, ...,   2,   0,   0],
         ...,
         [  3,   2, 126, ...,   2,   0,   0],
         [  5,   2,  20, ...,   2,   0,   0],
         [  7,   2,  20, ...,   2,   0,   0]], dtype=uint8),
  array([[0, 2],
         [1, 2],
         [3, 2],
         ...,
         [3, 2],
         [5, 2],
         [7, 2]], dtype=uint8)),
 '121:187:13.svbinttrc': (array([[ 0,  2,  0, ...,  2,  0,  0],
         [ 1,  2,  0, ...,  2,  0,  0],
         [ 3,  2,  0, ...,  2,  0,  0],
         ...,
         [ 9,  2, 50, ...,  2,  0,  0],
         [10,  2, 50, ...,  2,  0,  0],
         [11,  2, 50, ...,  2,  0,  0]], dtype=uint8),
  array([[ 0,  2],
         [ 1,  2],
         [ 3,  2],
         ...,
         [ 9,  2],
         [10,  2],
         [11,  2]], dtype=uint8)),
 '144:101:13.svbinttrc': (array([[  0,   2,   0, ...,   2,   0,   0],
         [  1,   2,   0, ...,   2

## Defining Neural Net

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
def format_label(l, width=16):
    val = bin(l)[2:]
    N = len(val)
    return [0.] * (width-N) + [float(i) for i in val]

In [6]:
format_label(512)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [7]:
features, labels = dict_dataset[list(dict_dataset.keys())[0]]
for file in list(dict_dataset.keys())[1:10]:
    features = np.append(features, dict_dataset[file][0], axis=0)
    labels = np.append(labels, dict_dataset[file][1], axis=0)

features_test, labels_test = dict_dataset[list(dict_dataset.keys())[10]]
for file in list(dict_dataset.keys())[11:13]:
    features_test = np.append(features_test, dict_dataset[file][0], axis=0)
    labels_test = np.append(labels_test, dict_dataset[file][1], axis=0)

In [8]:
#features, labels = dict_dataset[list(dict_dataset.keys())[0]]
#features_test, labels_test = dict_dataset[list(dict_dataset.keys())[1]]
#features = features - features.mean(axis=0)
# temp
features_temp = features#[9:15, ]
labels_temp = labels#[9:15]
# end temp

In [9]:
print(len(features_temp))
print(len(labels_temp))

10010
10010


In [10]:
print(labels_test[0:16])
print(labels_temp[0:16])


[[ 0  2]
 [ 1  2]
 [ 3  2]
 [ 5  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [12  2]
 [13  2]
 [14  2]
 [15  2]
 [16  2]
 [17  2]
 [ 3  2]
 [ 5  2]]
[[ 0  2]
 [ 1  2]
 [ 3  2]
 [ 5  2]
 [ 7  2]
 [ 8  2]
 [ 9  2]
 [10  2]
 [11  2]
 [12  2]
 [13  2]
 [14  2]
 [15  2]
 [16  2]
 [ 3  2]
 [ 5  2]]


In [11]:
labels_temp[2]

array([3, 2], dtype=uint8)

In [12]:
partial_mem_trace = features_temp[: , 0:600]
#partial_mem_trace = features_temp[: , 0:1024]
features_temp = partial_mem_trace # all rows are identical here

partial_mem_trace_test = features_test[: , 0:600]
#partial_mem_trace_test = features_test[: , 0:1024]
features_test = partial_mem_trace_test # all rows are identical here

    
print(partial_mem_trace.shape[0])
print(features_temp.shape[0])
print(partial_mem_trace.shape[1])
print(features_temp.shape[1])

10010
10010
600
600


In [13]:
features_temp[0]

array([  0,   2,   0,   0,   0,   0,  20,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

### Training Convolutional NN

In [14]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"

net = nn.Sequential(nn.Conv2d(1, 8, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(8, 16, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Conv2d(16, 32, kernel_size=(1,3), stride=1),
                    nn.ReLU(),
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Linear(19008, 2) #output layer: stays the same
                   )

#criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()

N_epochs = 10000

def validate(net, test_data, test_target, net_type=None):
    net = net.eval()
    
    with torch.no_grad():
        test_data_torch = torch.from_numpy(test_data).float().to(device)
        test_target_torch = torch.from_numpy(test_target).float().to(device)

        if net_type=='conv':
            test_data_torch = test_data_torch.unsqueeze(1).unsqueeze(2)
        
        test_pred = net(test_data_torch)
        print(test_pred.shape, test_target.shape)
        test_accuracy = (test_target_torch == test_pred).float().mean().to('cpu').detach().numpy()

        test_rmse = ((test_pred-test_target_torch)**2).mean().sqrt().to('cpu').detach().numpy()

    return test_accuracy, test_rmse

def train_net(net, train_data, train_target, test_data, test_target, N_epochs=10, print_freq=100, net_type=None):
    net = net.to(device)
    train_data_torch = torch.from_numpy(train_data).float().to(device)
    train_target_torch = torch.from_numpy(train_target).to(device)
    
    if net_type=='conv':
        train_data_torch = train_data_torch.unsqueeze(1).unsqueeze(2)
    
    net = net.train()
    for i in range(N_epochs):
        pred = net(train_data_torch.float())
        loss = criterion(pred, train_target_torch.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % print_freq == 0:
            train_accuracy, train_rmse = validate(net, train_data, train_target, net_type=net_type)
            test_accuracy, test_rmse = validate(net, test_data, test_target, net_type=net_type)
            net = net.train()
            print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')
            
    return net

#net = train_net(net, train_data, train_target, test_data, test_target, N_epochs=1000, print_freq=10)


In [15]:
optimizer = optim.Adam(net.parameters(), lr=1e-2)

In [16]:
#net = train(5000, net, features_temp, labels_temp, criterion, optimizer, debug=False)

#net(torch.from_numpy(features_temp[0]).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()).shape 

In [17]:
net = train_net(net, features_temp, labels_temp, features_test, labels_test, N_epochs=200, print_freq=10, net_type='conv')

torch.save(net, "conv_0")

torch.Size([10010, 2]) (10010, 2)
torch.Size([3003, 2]) (3003, 2)
Epoch = 0 loss=68.38848876953125 train_acc = 0.000 test_acc = 0.000 train_rmse = 50.089  test_rmse = 38.210


KeyboardInterrupt: 

In [ ]:
#net = net.to(device)
N_epochs = 10000

labels_temp = np.array(labels_temp)
#train_data_torch = torch.from_numpy(features_temp).float()
#train_target_torch = torch.from_numpy(labels_temp).float()
train_target_torch = torch.tensor(torch.from_numpy(np.array(labels_temp)).float())

import sys
np.set_printoptions(threshold=sys.maxsize)

def validate(net, test_data, test_target, i):
    
    test_data_torch = torch.from_numpy(features_test).float()
    #test_target_torch = torch.from_numpy(labels_test).float()
    test_target_torch = torch.tensor(torch.from_numpy(np.array(test_target)).float())

    test_pred = np.rint(net(test_data_torch).detach().numpy())
    if i % 500 == 0:
        print(np.concatenate((test_pred, test_target_torch.detach().numpy()), axis=1))
    #test_accuracy = (test_target_torch.detach().numpy() == test_pred).float().mean().detach().numpy()
    test_accuracy = (test_target_torch.detach().numpy() == test_pred).mean()

    #test_rmse = ((test_pred-test_target_torch)**2).mean().sqrt().detach().numpy()
    test_rmse = (np.sqrt((test_pred-test_target_torch.detach().numpy())**2).mean())

    return test_accuracy, test_rmse

for i in range(1):
    #pred = net(train_data_torch)
    #loss = criterion(pred, train_target_torch)
    
    #optimizer.zero_grad()
    #loss.backward()
    #optimizer.step()
    
    if i % 100 == 0:
        #train_accuracy, train_rmse = validate(net, features_temp, labels_temp)
        test_accuracy, test_rmse = validate(net, features_test, labels_test, i)
        #print(f'Epoch = {i} loss={loss} train_acc = {train_accuracy:.3f} test_acc = {test_accuracy:.3f} train_rmse = {train_rmse:.3f}  test_rmse = {test_rmse:.3f}')         
        print(f'Epoch = {i} test_acc = {test_accuracy:.3f}  test_rmse = {test_rmse:.3f}') 

In [ ]:
preds = net(torch.from_numpy(features_test[3]).float()).detach().numpy()
print(np.rint(preds))

In [ ]:
for p in preds:
    print(p)

In [60]:
labels_test[3]

array([5, 2], dtype=uint8)